In [1]:
import sys
from collections import defaultdict
from scipy.stats import kendalltau
import pandas as pd

sys.path.append("../")
from src.proxann.prompter import Prompter
from src.proxann.utils import process_responses, collect_fit_rank_data, compute_correlations_one

In [2]:
prompter = Prompter(model_type="gpt-4o-2024-08-06", temperature=0, seed=1234, config_path="../config/config.yaml")

src.proxann.prompter - INFO - Setting temperature to: 0
src.proxann.prompter - INFO - Setting seed to: 1234
src.proxann.prompter - INFO - Using OpenAI API with model: gpt-4o-2024-08-06


Loaded config file ../config/config.yaml and section logger.
Logs will be saved in data/logs
Loaded config file ../config/config.yaml and section llm.


In [58]:
template = """
System message:

Your input fields are:
1. `CATEGORY` (str)
2. `DOCUMENT` (str)

Your output fields are:
1. `reasoning` (str)
2. `FIT` (int): An integer score from 1 to 5 indicating how well the DOCUMENT fits the given CATEGORY

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## CATEGORY ## ]]
{CATEGORY}

[[ ## DOCUMENT ## ]]
{DOCUMENT}

[[ ## reasoning ## ]]
{reasoning}

[[ ## FIT ## ]]
{FIT}

[[ ## completed ## ]]

In adhering to this structure, your objective is:
        Determine whether the DOCUMENT fits with the given CATEGORY or not


User message:

[[ ## CATEGORY ## ]]
Relevant religious buildings

[[ ## DOCUMENT ## ]]
Alchi Monastery = Alchi Monastery or Alchi Gompa is a Buddhist monastery, known more as a monastic complex ( chos- ' khor ) of temples in Alchi village in the Leh District, of the Indian state under the Ladakh Autonomous Hill Development Council of Jammu and Kashmir. The complex comprises four separate settlements in the Alchi village in the lower Ladakh region with monuments dated to different periods. Of these four hamlets, Alchi monastery is said to be the oldest and most famous. It is administered by the Likir Monastery. Alchi is also part of the three villages ( all in lower Ladakh region ) which constitute the ‘ Alchi group of monuments ’ ; the other two villages adjoining Alchi are the Mangyu and Sumda Chun.

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## FIT ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Assistant message:

[[ ## reasoning ## ]]
The document describes Alchi Monastery, which is a Buddhist monastery and a significant religious building in the Leh District of India. It provides details about its historical importance, architectural features, and its role within the local religious community. Since the category is "Relevant religious buildings," and the document discusses a specific religious site, it fits well within this category.

[[ ## FIT ## ]]
5

[[ ## completed ## ]]


User message:

[[ ## CATEGORY ## ]]
America professional basketball

[[ ## DOCUMENT ## ]]
2007 NBA betting scandal = The 2007 NBA betting scandal was a scandal involving the National Basketball Association ( NBA ) and accusations that an NBA referee used his knowledge of relationships between referees, coaches, players and owners to bet on professional basketball games. In July 2007, reports of an investigation by the Federal Bureau of Investigation ( FBI ) were made public, which alleged that during the 2005 – 06 and 2006 – 07 NBA seasons, referee Tim Donaghy bet on games in which he officiated. On August 15, 2007, Donaghy pleaded guilty to two federal charges related to the investigation, and a year later he was sentenced to 15 months in prison and three years of supervised release.

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## FIT ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Assistant message:

[[ ## reasoning ## ]]
The document discusses the 2007 NBA betting scandal, which directly involves the National Basketball Association (NBA) and its referees, players, and the integrity of professional basketball games. This topic is highly relevant to the category of "America professional basketball" as it pertains to a significant event in the history of the league and highlights issues related to gambling and officiating in professional basketball. Therefore, the document fits well within the category.

[[ ## FIT ## ]]
4

[[ ## completed ## ]]


User message:

[[ ## CATEGORY ## ]]
Japanese and Korean Visual Novels and Manga Series

[[ ## DOCUMENT ## ]]
To Heart 2 = To Heart 2 ( トゥハート２, Tu Hāto 2 ), stylized as ToHeart2, is a Japanese romance visual novel developed by Leaf and published by Aquaplus. It was first released for the PlayStation 2 on December 28, 2004 as an all-ages title, and was followed by an adult version playable on Microsoft Windows and subsequent all-ages versions for the PlayStation Portable and PlayStation 3. This deviated from the release history of the game 's predecessor, To Heart, which was originally released with adult content prior to receiving versions with such content removed.

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## FIT ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.
"""

In [65]:
template = """
Please act as an impartial judge and assign an integer score from 1 to 5 indicating how well the DOCUMENT fits the given CATEGORY. Do not provide any reasoning or explanation

[[ ## CATEGORY ## ]]
Japanese and Korean Visual Novels and Manga Series

[[ ## DOCUMENT ## ]]
To Heart 2 = To Heart 2 ( トゥハート２, Tu Hāto 2 ), stylized as ToHeart2, is a Japanese romance visual novel developed by Leaf and published by Aquaplus. It was first released for the PlayStation 2 on December 28, 2004 as an all-ages title, and was followed by an adult version playable on Microsoft Windows and subsequent all-ages versions for the PlayStation Portable and PlayStation 3. This deviated from the release history of the game 's predecessor, To Heart, which was originally released with adult content prior to receiving versions with such content removed.
"""

In [66]:
response_q2, logprobs = prompter.prompt(None, template, use_context=False, temperature=0)

Cache key: e918d19710229453583f8ed452a07cad


In [67]:
logprobs[0].top_logprobs

[TopLogprob(token='5', bytes=[53], logprob=-0.014185127802193165),
 TopLogprob(token='4', bytes=[52], logprob=-4.264184951782227),
 TopLogprob(token='3', bytes=[51], logprob=-10.764184951782227),
 TopLogprob(token='1', bytes=[49], logprob=-14.889184951782227),
 TopLogprob(token='2', bytes=[50], logprob=-15.889184951782227),
 TopLogprob(token='Score', bytes=[83, 99, 111, 114, 101], logprob=-16.764184951782227),
 TopLogprob(token='The', bytes=[84, 104, 101], logprob=-17.889184951782227),
 TopLogprob(token=' ', bytes=[32], logprob=-18.514184951782227),
 TopLogprob(token='[[', bytes=[91, 91], logprob=-19.139184951782227),
 TopLogprob(token='6', bytes=[54], logprob=-20.014184951782227),
 TopLogprob(token='Category', bytes=[67, 97, 116, 101, 103, 111, 114, 121], logprob=-20.389184951782227),
 TopLogprob(token='\n', bytes=[10], logprob=-20.514184951782227),
 TopLogprob(token='<|end|>', bytes=None, logprob=-20.514184951782227),
 TopLogprob(token='Five', bytes=[70, 105, 118, 101], logprob=-20.7

In [ ]:
import math

tokens_probs = []
word_to_number = {
    "uno": 1,
    "dos": 2,   
    "tres": 3,
    "cuatro": 4,
    "cinco": 5
}

print(len(logprobs[0].top_logprobs))
for top_logprobs in logprobs[0].top_logprobs:
    raw_token = top_logprobs.token
    try:
        token_str = str(raw_token).lower()
        token = word_to_number.get(token_str, int(raw_token))
    except (ValueError, TypeError):
        print(f"Token: {raw_token} could not be converted to int")
        continue

    if token in {1, 2, 3, 4, 5}:  # set is faster for membership tests
        prob = math.exp(top_logprobs.logprob)
        tokens_probs.append((token, prob))

# final score
score = sum(token * prob for token, prob in tokens_probs)
score

20
Token: Score could not be converted to int
Token: The could not be converted to int
Token:   could not be converted to int
Token: [[ could not be converted to int
Token: Category could not be converted to int
Token: 
 could not be converted to int
Token: <|end|> could not be converted to int
Token: Five could not be converted to int
Token: I could not be converted to int
Token: Assign could not be converted to int
Token: CATEGORY could not be converted to int
Token: To could not be converted to int
Token: Rating could not be converted to int


4.985892358591917

In [37]:
data_jsons = [
    "/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/data_used_in_paper/json_out/config_bills_part1.json",
    "/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/data_used_in_paper/json_out/config_bills_part2.json",
    "/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/data_used_in_paper/json_out/config_pilot_wiki_part2.json",
    "/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/data_used_in_paper/json_out/config_pilot_wiki.json"
]
response_csvs = [
    "/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/data_used_in_paper/qualtrics/Cluster+Evaluation+-+Sort+and+Rank_December+12,+2024_05.19.csv",
    "/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/data_used_in_paper/qualtrics/Cluster+Evaluation+-+Sort+and+Rank+-+Bills_December+14,+2024_13.20.csv"
]

llm_data_paths = {
    "wiki": {
        "gpt4o-mini-bin": "/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/data_used_in_paper/llm_results/wiki/q1_then_q3_dspy,q1_then_q2_dspy_gpt-4o-mini-2024-07-18_20241213_102655",
        
        "gpt4o-mini-non-bin": "/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/paper_wang/wiki/q1_then_q2_dspy,q1_then_q3_dspy_gpt-4o-mini-2024-07-18_20250415_163508",
       
        "gpt4o-non-bin": "/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/paper_wang/wiki/q1_then_q2_dspy,q1_then_q3_dspy_gpt-4o-2024-08-06_20250415_164926",
    }
}

cohr_path = ["/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/cohrs/all_cohrs_bills.csv", "/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/cohrs/all_cohrs_wiki.csv"]

start_date = "2024-12-06 09:00:00"

In [38]:
responses = {}
for csv in response_csvs:
    for topic_id, topic_responses in process_responses(csv, data_jsons, start_date=start_date, path_save=None, removal_condition="loose").items():
        if topic_responses:
            responses[topic_id] = topic_responses

_, _, _, corr_data = collect_fit_rank_data(responses)

Total responses: 142
Total responses: 142
Removed: 25
Total responses: 121
Total responses: 121
Removed: 20


In [39]:
# read coherence metrics
cohrs = pd.concat([pd.read_csv(cohr_path[0]), pd.read_csv(cohr_path[1])]).to_dict(orient="records")

corr_ids = [x["id"] for x in corr_data]
corr_metric = "agree" #"tau"
agg = "mean"

for corr_metric in ["tau"]:#, "agree"

    # Obtener las claves internas del primer nivel del diccionario
    row_names = ["npmi", "cv"] + \
    [f"{metric}_{llm}" for llm in list(llm_data_paths[list(llm_data_paths.keys())[0]].keys()) for metric in ["fit", "rank"]]

    column_names = [f"{metric}_{dataset}" for dataset in ["wiki"] for metric in ["fit", "rank"]]

    results_df = pd.DataFrame(index=row_names, columns=column_names)

    corrs_mode_1 = defaultdict(dict)
    # Fill the DataFrame with calculated values
    for dataset in ["wiki"]:
        print(f"Processing {dataset} dataset")
        llm_fits, llm_ranks = {}, {}
        for llm, path in llm_data_paths[dataset].items():
            fits = pd.read_json(f"{path}/llm_results_q2.json").to_dict(orient="records")
            ranks = pd.read_json(f"{path}/llm_results_q3.json").to_dict(orient="records")
            for item in fits:
                item["annotators"] = [llm]
            for item in ranks:
                item["annotators"] = [llm]
            llm_fits[llm] = fits
            llm_ranks[llm] = ranks
        
        for llm in llm_data_paths[dataset]:  # Iterate through each LLM
            
            dtset_corr_ids = [x for x in corr_ids if dataset in x]
        
            filtered_ranks = [x for x in llm_ranks[llm] if x["id"] in dtset_corr_ids]
            filtered_fits = [x for x in llm_fits[llm] if x["id"] in dtset_corr_ids]
            filtered_cohrs = [x for x in cohrs if x["id"] in dtset_corr_ids] 
            filtered_corr_data = [x for x in corr_data if x["id"] in dtset_corr_ids]
            
            # ensure that the lists are sorted by id        
            filtered_ranks = sorted(filtered_ranks, key=lambda x: x["id"])
            filtered_fits = sorted(filtered_fits, key=lambda x: x["id"])
            filtered_cohrs = sorted(filtered_cohrs, key=lambda x: x["id"])
            filtered_corr_data = sorted(filtered_corr_data, key=lambda x: x["id"])
            
            ids_fits = [x["id"] for x in filtered_fits]
            ids_ranks = [x["id"] for x in filtered_ranks]
            ids_corr = [x["id"] for x in filtered_corr_data]
                    
            # check equality of ids
            assert ids_fits == ids_ranks == ids_corr
                    
            npmi_cohrs = [x["npmi"] for x in filtered_cohrs]
            cv_cohrs = [x["cv"] for x in filtered_cohrs]

            #import pdb; pdb.set_trace()
            corrs_mode1_llm = compute_correlations_one(filtered_corr_data, filtered_ranks, filtered_fits, aggregation_method=agg)
            
            corrs_mode_1[dataset][llm] = corrs_mode1_llm

            for user_metric in ["fit", "rank"]:#, "rank"]
                user_tm_metric = corrs_mode1_llm[f"{user_metric}_{corr_metric}"]

                # Correlate user_tm_metric with npmi and cv
                npmi_user_metric = kendalltau(npmi_cohrs, user_tm_metric, nan_policy="omit").statistic
                cv_user_metric = kendalltau(cv_cohrs, user_tm_metric, nan_policy="omit").statistic

                # Store the npmi and cv correlations in the DataFrame
                results_df.loc["npmi", f"{user_metric}_{dataset}"] = npmi_user_metric
                results_df.loc["cv", f"{user_metric}_{dataset}"] = cv_user_metric

                for llm_metric in ["fit", "rank"]: #, "rank"
                    llm_tm_metric = corrs_mode1_llm[f"{llm_metric}_{corr_metric}_tm_{llm}"]
                    
                    # Correlate user_tm_metric with llm_tm_metric
                    tau_topic_rank = kendalltau(llm_tm_metric, user_tm_metric, nan_policy="omit").statistic

                    # Store the topic ranking correlation in the DataFrame
                    results_df.loc[f"{llm_metric}_{llm}", f"{user_metric}_{dataset}"] = tau_topic_rank

    print(f"TAU of {corr_metric}")
    print(results_df)
    print("\n")

Processing wiki dataset
TAU of tau
                         fit_wiki rank_wiki
npmi                     0.029091 -0.122022
cv                      -0.036364 -0.203369
fit_gpt4o-mini-bin       0.455181  0.182271
rank_gpt4o-mini-bin      0.451972  0.447997
fit_gpt4o-mini-non-bin   0.153316  0.106467
rank_gpt4o-mini-non-bin  0.033916  0.114956
fit_gpt4o-non-bin        0.311568  0.423692
rank_gpt4o-non-bin       0.203891  0.307148




In [43]:
columns_show = [col for col in corrs_mode_1["wiki"]["gpt4o-mini-bin"].columns if "rank" in col]
print(corrs_mode_1["wiki"]["gpt4o-mini-bin"].aggregate(lambda x: x.mean() if x.dtype == "float64" else x.mode()[0], axis=0)[columns_show])

print("\n")

columns_show = [col for col in corrs_mode_1["wiki"]["gpt4o-mini-non-bin"].columns if "rank" in col]
print(corrs_mode_1["wiki"]["gpt4o-mini-non-bin"].aggregate(lambda x: x.mean() if x.dtype == "float64" else x.mode()[0], axis=0)[columns_show])

print("\n")

columns_show = [col for col in corrs_mode_1["wiki"]["gpt4o-non-bin"].columns if "rank" in col]
print(corrs_mode_1["wiki"]["gpt4o-non-bin"].aggregate(lambda x: x.mean() if x.dtype == "float64" else x.mode()[0], axis=0)[columns_show])

rank_rho                           0.50885
rank_tau                          0.422711
rank_ndcg                         0.843065
rank_rho_users_gpt4o-mini-bin     0.654334
rank_rho_tm_gpt4o-mini-bin        0.469563
rank_tau_users_gpt4o-mini-bin     0.544803
rank_tau_tm_gpt4o-mini-bin        0.355845
rank_ndcg_users_gpt4o-mini-bin    0.904824
rank_ndcg_tm_gpt4o-mini-bin        0.73166
dtype: object


rank_rho                               0.50885
rank_tau                              0.422711
rank_ndcg                             0.843065
rank_rho_users_gpt4o-mini-non-bin     0.527159
rank_rho_tm_gpt4o-mini-non-bin        0.529075
rank_tau_users_gpt4o-mini-non-bin     0.435986
rank_tau_tm_gpt4o-mini-non-bin        0.408998
rank_ndcg_users_gpt4o-mini-non-bin    0.864052
rank_ndcg_tm_gpt4o-mini-non-bin       0.765549
dtype: object


rank_rho                          0.50885
rank_tau                         0.422711
rank_ndcg                        0.843065
rank_rho_users_gpt4o-non-bin    

In [41]:
columns_show = [col for col in corrs_mode_1["wiki"]["gpt4o-mini-bin"].columns if "fit" in col]
print(corrs_mode_1["wiki"]["gpt4o-mini-bin"].aggregate(lambda x: x.mean() if x.dtype == "float64" else x.mode()[0], axis=0)[columns_show])

print("\n")

columns_show = [col for col in corrs_mode_1["wiki"]["gpt4o-mini-non-bin"].columns if "fit" in col]
print(corrs_mode_1["wiki"]["gpt4o-mini-non-bin"].aggregate(lambda x: x.mean() if x.dtype == "float64" else x.mode()[0], axis=0)[columns_show])

print("\n")

columns_show = [col for col in corrs_mode_1["wiki"]["gpt4o-non-bin"].columns if "fit" in col]
print(corrs_mode_1["wiki"]["gpt4o-non-bin"].aggregate(lambda x: x.mean() if x.dtype == "float64" else x.mode()[0], axis=0)[columns_show])

fit_rho                            0.51491
fit_tau                           0.427663
fit_agree                         0.761905
fit_ndcg                          0.888187
fit_tau_users_gpt4o-mini-bin      0.540727
fit_tau_tm_gpt4o-mini-bin         0.400193
fit_agree_users_gpt4o-mini-bin    0.422619
fit_agree_tm_gpt4o-mini-bin       0.339286
dtype: object


fit_rho                                0.51491
fit_tau                               0.427663
fit_agree                             0.761905
fit_ndcg                              0.888187
fit_tau_users_gpt4o-mini-non-bin       0.51821
fit_tau_tm_gpt4o-mini-non-bin          0.37233
fit_agree_users_gpt4o-mini-non-bin    0.738095
fit_agree_tm_gpt4o-mini-non-bin       0.690476
dtype: object


fit_rho                           0.51491
fit_tau                          0.427663
fit_agree                        0.761905
fit_ndcg                         0.888187
fit_tau_users_gpt4o-non-bin      0.554961
fit_tau_tm_gpt4o-non-bin         0.404